# Instacart Market Basket Analysis

In [1]:
import csv
import numpy as np
import pandas as pd
from zipfile import ZipFile
from collections import Counter
from sklearn.neighbors import LSHForest
from scipy.sparse import csr_matrix, vstack
from sklearn.model_selection import train_test_split

In [2]:
'''This method opens the category specific zip file and returns a pandas data frame of the csv file contained within'''
def extraction(category):
    csv = category + '.csv'
    z = csv + '.zip'
    zf = ZipFile(z, 'r')
    df = pd.read_csv(zf.open(csv))
    return df

aisles = extraction('aisles')
departments = extraction('departments')
order_products_prior = extraction('order_products__prior')
order_products_train = extraction('order_products__train')
orders = extraction('orders')
products = extraction('products')

In [3]:
# This data frame tells to which set (prior, train, test) an order belongs. 
# The goal is to predict reordered items only for the test set orders.
# User 1 has 10 previous (prior) orders which will be used to predict items in his final (train) order.
orders.head(11)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [4]:
# order_products_prior and order_products_train specify which products were purchased in each order. 
# order_products_prior contains previous order contents for all customers. 
# order_products_train contains the final order contents for those users in the training set.
# 'reordered' indicates that the customer has a previous order that contains the product.
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
# Products of last order for each user
train_orders = pd.merge(orders[orders['eval_set']=='train'], order_products_train, on='order_id', how='left')
prior_orders = pd.merge(orders[orders['eval_set']=='prior'], order_products_prior, on='order_id', how='left')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [6]:
print('Unique users in entire training set: {0}'.format(train_orders['user_id'].nunique()))
print('Unique users in testing set: {0}'.format(prior_orders['user_id'].nunique()-train_orders['user_id'].nunique()))
print('Total unique users: {0}'.format(prior_orders['user_id'].nunique()))

Unique users in entire training set: 131209
Unique users in testing set: 75000
Total unique users: 206209


In [7]:
# Set of users for training
keep = train_orders['user_id'].unique()

# Prior orders of training set

In [8]:
# prior_orders contains all user ids and we only need those in the training set for now
prior_orders_keep = prior_orders[prior_orders['user_id'].isin(keep)]
prior_orders_keep.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [9]:
prior_orders_keep['user_id'].nunique()

131209

# Prior orders of testing set

In [10]:
# Constructs a data frame that uses users only in the testing set
prior_orders_test = prior_orders[~prior_orders['user_id'].isin(keep)]
prior_orders_test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
254,1374495,3,prior,1,1,14,NaN,9387,1,0
255,1374495,3,prior,1,1,14,NaN,17668,2,0
256,1374495,3,prior,1,1,14,NaN,15143,3,0
257,1374495,3,prior,1,1,14,NaN,16797,4,0
258,1374495,3,prior,1,1,14,NaN,39190,5,0


In [11]:
prior_orders_test['user_id'].nunique()

75000

# Getting order_id's for submission

In [12]:
# Only look at order information for those in the test set
orders[orders['eval_set']=='test'].head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [13]:
# From those orders a list is made of the order_id's
test_order_ids = list(orders[orders['eval_set']=='test']['order_id'])
test_order_ids[:5]

[2774568, 329954, 1528013, 1376945, 1356845]

# Final orders of users in training set

In [14]:
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


# Create Sparse Matrices

In [15]:
prior_orders_keep.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [16]:
# The goal is to get the counts for each product bought by each user. 
# As such we need only the user_id and product_id columns
prior_orders_keep = prior_orders_keep[['user_id', 'product_id']]
train_orders = train_orders[['user_id', 'product_id']]
test_orders = prior_orders_test[['user_id', 'product_id']]

prior_orders_keep.head()

,user_id,product_id
0,1,196
1,1,14084
2,1,12427
3,1,26088
4,1,26405


In [17]:
test_orders.head()

,user_id,product_id
254,3,9387
255,3,17668
256,3,15143
257,3,16797
258,3,39190


In [18]:
# This makes the data frame a list of list with each inner list consisting of two items.
# The first is user_id and the second is product_id.
prior_orders_list = prior_orders_keep.values.tolist()
train_orders_list = train_orders.values.tolist()
test_orders_list = test_orders.values.tolist()

In [19]:
prior_orders_list[:5]

[[1, 196], [1, 14084], [1, 12427], [1, 26088], [1, 26405]]

In [20]:
'''Creates a compressed sparse row matrix with columns corresponding to user_ids and rows corresponding to product_ids.
   The intesecting cells are the total number of a specific product that a specific user has bought in their previous 
   orders.'''
def csr_gen(order_list):
    indptr = [0]
    indices = []
    data = []
    cur_usr = order_list[0][0]
    cur_len = 0
    count = 1
    for tpl in order_list:
        if tpl[0] != cur_usr:
            indptr.append(len(indices))
            cur_usr = tpl[0]
            count+= 1
        indices.append(tpl[1]-1)
        data.append(1)
    # Makes sure the last case is accounted for    
    indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int)#.toarray()

In [21]:
prior_matrix = csr_gen(prior_orders_list)
train_matrix = csr_gen(train_orders_list)
test_matrix = csr_gen(test_orders_list)

In [22]:
# Product info for user 1 in prior
Counter(prior_matrix.indices[prior_matrix.indptr[0]:prior_matrix.indptr[1]])

Counter({195: 10,
         10257: 9,
         10325: 1,
         12426: 10,
         13031: 3,
         13175: 2,
         14083: 1,
         17121: 1,
         25132: 8,
         26087: 2,
         26404: 2,
         30449: 1,
         35950: 1,
         38927: 1,
         39656: 1,
         41786: 1,
         46148: 3,
         49234: 2})

In [23]:
print('Items bought in prior_matrix: {0}'.format(prior_matrix.nnz))
print('Items bought in train_matrix: {0}'.format(train_matrix.nnz))
print('Items bought in test_matrix: {0}'.format(test_matrix.nnz))

Items bought in prior_matrix: 20641991
Items bought in train_matrix: 1384617
Items bought in test_matrix: 11792498


# Model Analysis

In [24]:
'''This class is used to evaluate LSHForest models with varying parameters to find an optimal model and to then predict
   the re-ordered products in the final orders of the testing set'''
class instacart:
    
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.cv = False
        

    '''Performs k-fold cross validation of an LSHForest estimator. self.cv is set to True so that in the model_eval
       method the training and testing matrices are not redefined and that the f1 scores are calculated and stored in
       in self.f1s. The mean and standard deviation of these scores are stored in self.f1_mean and self.f1_std. self.cv
       is set to false once cross validation has completed in order to allow model_eval to be used without having to
       create another instance of the class.'''
    def lshf_cross_val(self, k, n_est=10, n_cand=50, n_neigh=5, thresh=0.5):
        self.cv = True
        self.f1s = []
        self.cv_params = [k, n_est, n_cand, n_neigh, thresh]
        # For random partitioning of the data into X_sets and y_sets
        self.X_sets = []
        self.y_sets = []
        index = np.arange(np.shape(self.X)[0])  # Creates a vector of all the indices
        np.random.shuffle(index)  # Shuffles their ordering
        index = np.array_split(index, k)  # Splits them into appoximately equal subsets (equal if len(index)%k==0)
        for subset in index:
            self.X_sets.append(self.X[subset, :])
            self.y_sets.append(self.y[subset, :])
        # Performs k fold cross validation obtaining mean f1 scores over the orders in the test set
        for x in range(k):
            self.X_test = self.X_sets[x]
            self.y_test = self.y_sets[x]
            self.X_train = vstack(self.X_sets[:x] + self.X_sets[(x+1):])
            self.y_train = vstack(self.y_sets[:x] + self.y_sets[(x+1):])
            self.model_eval(n_estimators=n_est, n_candidates=n_cand, n_neighbors=n_neigh, threshold=thresh)
        # Calculates the average mean and standard deviation of the results from the folds   
        self.f1_mean = np.mean(self.f1s)
        self.f1_std = np.std(self.f1s)
        self.cv = False
    
   
    '''Evaluates an LSHForest model. n_estimators is the number of trees in the forest. n_candidates is the minimum 
       number of candidates evaluated per estimator, assuming enough items meet the min_hash_match contraint (set to 4
       inside the method). n_neighbors is the number of neighbors to be returned for each user. threshold is used to
       determine if an item should be predicted in the test users next order or not. For example, threshold being 0.5
       and n_neighbors=10 indicates that an item must be in at least 0.5*10 = 5 of the nearest neighbors previous order
       history in order to be predicted. test_set is None except when determining the predictions for the kaggle test
       set (test_matrix). train_score is False except when trying to determine the how well the model performs on the
       set that trained it.'''
    def model_eval(self, n_estimators, n_candidates, n_neighbors, threshold=0.5, test_set=None, train_score=False):
       
        if (test_set!=None) & train_score:
            raise ValueError('If a test_set is used, train_score must be False')
        
        self.preds = []  # Makes repeated use of the class instance possible
        
        if self.cv==False:
            if train_score:
                self.f1s = []
                test_set = self.X
            self.X_train, self.y_train, self.X_test = self.X, self.y, test_set
        
        # Does fitting a model make it forget previous fits?
        self.lshf = LSHForest(n_estimators=n_estimators,
                    radius=1.0,
                    n_candidates=n_candidates,
                    n_neighbors=n_neighbors,
                    min_hash_match=4,
                    radius_cutoff_ratio=0.9,
                    random_state=None)
        
#         print('Fitting the model')
        self.lshf.fit(self.X_train, self.y_train)
        
#         print('Finding {0} approximate nearest neighbors per user'.format(n_neighbors))
        # sim_users is a matrix of shape (self.X_test.shape[0], n_neighbors) where each row is a vector corresponding
        # to a user in the testing set where the entries in the vector are the indices of their approximate nearest
        # neighbors in training set
        sim_users = self.lshf.kneighbors(self.X_test, return_distance=False)
#         print('Neighbors found')
        
        # prod_set is a list of lists where each list inside of it consists of the set of items for each user in
        # a row of sim_users
        prod_set = []
        for usr_set in sim_users:
            set_prods = []  # List of items bought by the approximate neighbors closest users
            for usr in usr_set:
                # Using set here makes sure that each item bought by the neighbors is only counted once
                # when determining which ones should be predicted 
                set_prods.extend(list(set(self.X_train[usr].indices)))
            prod_set.append(set_prods)    
                    
        for prods in prod_set:
            # If the item is bought by least half of the most similar users it is predicted in the next order
            key_items = [key for key, count in Counter(prods).items() if count>=n_neighbors*threshold]
            self.preds.append(key_items)
        
        # Used when performing cross validation on the training set
        if self.cv==True:
            self.f1s.append(f1_score(self.preds, self.y_test))
#             print(self.f1s)
            return
        # Non cross validation
        else:
            # Used when just generating predictions
            # Changes the empty predictions for users into the string 'None' (Part of submission reqs for Kaggle)
            if train_score==False:
                return
            # Get training accuracy
            else:
                self.f1s.append(f1_score(self.preds, self.y_test))
#                 print(self.f1s)
                return   

In [25]:
'''Uses the predictions (preds) and actuals (y_test) to determine the mean f1 score across all final orders'''
def f1_score(preds, y_test):
    
    f1s = []  # List of f1 scores per user in the testing set
    for row in range(y_test.shape[0]):
        expected_correct = len(y_test[row].indices)
        TP = 0
        FP = 0
        for item in preds[row]:
            if item in y_test[row].indices:
                TP +=1  # If the predicted item is in the actual, the number of true positives increase
            else:
                FP += 1  # Otherwise the number of false positives increase
        if TP==0: 
            if expected_correct==0: 
                f1 = 1.0  # 0 true positives and 0 elements in row indicates a correct prediction
            else:
                f1 = 0.0  # Otherwise it's wrong entirely
        else:
            precision = TP/(TP+FP)
            recall = TP/expected_correct
            f1 = 2*(precision*recall)/(precision+recall)
        f1s.append(f1)
    mean_f1 = np.mean(f1s)
    return mean_f1


'''Creates a submission file for Kaggle based in their desired format.'''
def submission(test_order_ids, preds):
    
    new_preds = []
    count = 0
    for user in preds:
        usr = []
        for pred in user:
            if pred in set(test_matrix[count].indices):
                usr.append(pred)
        new_preds.append(usr)
        count += 1

    f = open('instacart_submission.csv', 'w')
    try:
        writer = csv.writer(f)
        writer.writerow(('order_id', 'products'))
        i = 0
        for pred in new_preds:
            if pred==[]:
                writer.writerow((test_order_ids[i], 'None'))
                i += 1
            else:
                items = ''
                for item in pred:
                    items += str(item) + ' '
                writer.writerow((test_order_ids[i], items))
                i += 1
    finally:
        f.close()

In [ ]:
kaggle = instacart(X=prior_matrix, y=train_matrix)

In [ ]:
kaggle.lshf_cross_val(k=3, n_est=50, n_cand=200, n_neigh=10, thresh=0.5)

In [28]:
(kaggle.f1_mean, kaggle.f1_std)

(0.15447683550555358, 0.00064333538784715303)

In [29]:
kaggle.model_eval(n_estimators=50, n_candidates=200, n_neighbors=10, test_set=test_matrix)

In [30]:
submission(test_order_ids=test_order_ids, preds=kaggle.preds)